Example of how to use the `collecting_tweets.py` to generate the required dataframes. Futhermore, `lat` & `lng` columns are generated in places dataframe.

In [1]:
import pandas as pd
import collecting_tweets
from geopy.geocoders import Nominatim
import os

from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

In [2]:
# to get city coordinates
geolocator = Nominatim(user_agent = 'bmartin')

In [3]:
# loadinng credenttials as environmen variables
load_dotenv('data/envs/kafka.env', override = True)


# getting twitter credentials
twitter_key = os.environ.get('api_key')
twitter_secret_key = os.environ.get('secret_key')
bearer_token = os.environ.get('bearer_token')

In [4]:
# Get daa
search_tweet = collecting_tweets.search_tweets(query='covid', bearer_token = bearer_token, next_token = None)

# Check if we have tweet's location
if "places" in search_tweet['includes'].keys():
    main_tweets, main_users, main_places = collecting_tweets.create_dataframes(search_tweet)
    
else:
    main_tweets, main_users = collecting_tweets.create_dataframes(search_tweet)
    main_places = pd.DataFrame()

In [5]:
main_places

""


In [6]:
# Generate dataframes
try:
    main_tweets, main_users, main_places = collecting_tweets.more_tweets(20, "covid", search_tweet,  main_tweets, 
                                                                         main_users, main_places)

except ValueError:
    main_tweets, main_users = collecting_tweets.more_tweets(20, "covid", search_tweet, main_tweets, 
                                                            main_users, main_places)


In [7]:
main_places.head()

,country,full_name,geo.place_id,name
0,Austria,"Viena, Austria",9f659d51e5c5deae,Viena
1,Polinesia Francesa,Polinesia Francesa,ea5eca163987d547,Polinesia Francesa
2,Estados Unidos,"Spokane, WA",dc3747428fa88cab,Spokane
3,Brasil,"Navidad, Brasil",a4ddc3856053f7e1,Navidad
4,Guatemala,Guatemala,13d479b108707983,Guatemala


In [8]:
if main_places.empty:
    pass

else:
    main_places["lat"] = main_places.full_name.apply(lambda x: geolocator.geocode(x)[1][0])
    main_places["lng"] = main_places.full_name.apply(lambda x: geolocator.geocode(x)[1][1])

In [9]:
main_places.head()

,country,full_name,geo.place_id,name,lat,lng
0,Austria,"Viena, Austria",9f659d51e5c5deae,Viena,48.208354,16.372504
1,Polinesia Francesa,Polinesia Francesa,ea5eca163987d547,Polinesia Francesa,-16.034425,-146.049093
2,Estados Unidos,"Spokane, WA",dc3747428fa88cab,Spokane,47.657193,-117.423511
3,Brasil,"Navidad, Brasil",a4ddc3856053f7e1,Navidad,-17.786042,-63.155888
4,Guatemala,Guatemala,13d479b108707983,Guatemala,15.687101,-90.122655


In [10]:
main_tweets

,created_at,tweet_id,text,lang,possibly_sensitive,type
0,2021-12-02T20:30:12.000Z,1466504999773646853,@pfizer Why are you secretly adding Tromethami...,en,False,replied_to
1,2021-12-02T20:30:12.000Z,1466504999689760775,Internment camps \nWe have become animals\n#CO...,en,False,None
2,2021-12-02T20:30:12.000Z,1466504999291310088,https://t.co/SPFXjj8h8B,und,False,None
3,2021-12-02T20:30:12.000Z,1466504999098544129,‘We need to be ready’ — Biden unveils COVID-19...,en,False,None
4,2021-12-02T20:30:12.000Z,1466504998355881989,Situación COVID-19 al 2/12\n\nHoy 2.681 nuevos...,es,False,None
...,...,...,...,...,...,...
620,2021-12-02 20:28:52+00:00,1466504660525752326,@TPostMillennial Government + Science= $ Paid ...,en,False,replied_to
621,2021-12-02 20:28:51+00:00,1466504659829329920,getting my flu shot and covid booster on satur...,en,False,None
622,2021-12-02 20:28:51+00:00,1466504659678474245,Venezuela reporta 569 nuevos casos por Covid-1...,es,False,None
623,2021-12-02 20:28:51+00:00,1466504659020001288,"Los síntomas de la nueva variante de Covid-19,...",es,False,None
